In [1]:
#Import data manipulation libaries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
import datetime, lightgbm, math, catboost, xgboost, warnings

#Models
from sklearn.base import clone
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier, VotingClassifier
from sklearn.metrics import log_loss, mean_squared_log_error
from sklearn.calibration import CalibrationDisplay, CalibratedClassifierCV
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier, XGBRegressor

#Splitting the data and cross-validation
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, cross_val_score,KFold ,GroupKFold, StratifiedKFold

#Metrics
#from sklearn.metrics import classification_report,accuracy_score , roc_auc_score
from sklearn.metrics import mean_squared_error, roc_auc_score, log_loss, mean_squared_log_error
#Classifiers

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

#Feature engineerring
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder, LabelEncoder, PolynomialFeatures
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer

#Encoding

import category_encoders as ce

#from pygam import GAM, LogisticGAM, s, f, te, l

np.set_printoptions(linewidth=150, edgeitems=5)

In [3]:
train = pd.read_csv("/kaggle/input/playground-series-s3e11/train.csv")
test= pd.read_csv("/kaggle/input/playground-series-s3e11/test.csv")
submission = pd.read_csv("/kaggle/input/playground-series-s3e11/sample_submission.csv")
original = pd.read_csv('/kaggle/input/media-campaign-cost-prediction/train_dataset.csv')

In [4]:
sets={'train':train,'test':test,'original':original}

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360336 entries, 0 to 360335
Data columns (total 17 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          360336 non-null  int64  
 1   store_sales(in millions)    360336 non-null  float64
 2   unit_sales(in millions)     360336 non-null  float64
 3   total_children              360336 non-null  float64
 4   num_children_at_home        360336 non-null  float64
 5   avg_cars_at home(approx).1  360336 non-null  float64
 6   gross_weight                360336 non-null  float64
 7   recyclable_package          360336 non-null  float64
 8   low_fat                     360336 non-null  float64
 9   units_per_case              360336 non-null  float64
 10  store_sqft                  360336 non-null  float64
 11  coffee_bar                  360336 non-null  float64
 12  video_store                 360336 non-null  float64
 13  salad_bar     

In [6]:
#Merge train and original dataset
train = pd.concat([train, original], axis = 0)

In [7]:
#Create labels and starting feature set

# Define the target
target = "cost"
# Defibe the features before feture engineering
original_features = list(test.columns)
# Define the training labels
train_y = train[target]
#Define the starting training features
train_X = train.drop(target , axis = 1)

In [8]:
train_X.shape,  train_y.shape, type(train_y)

((411699, 16), (411699,), pandas.core.series.Series)

In [9]:
# Add features

train_X= train_X.copy()
test = test.copy()

# Define the list of dataframe to which to apply the transformations
dataset = [train_X, test]

features_facilities = ['coffee_bar', 'video_store', 'salad_bar', 'prepared_food', 'florist']

for df in dataset:
    df['sales_per_unit'] = df['store_sales(in millions)'] / df['unit_sales(in millions)']
    df['sales_per_sqft'] = df['store_sales(in millions)'] / df['store_sqft']
#    df['cars_per_households'] = df['avg_cars_at home(approx).1'] / (df['num_children_at_home'] + 2)
    df['has_all_facilities'] = df[features_facilities].all(axis=1)
    df['facilities_rate'] = df[features_facilities].sum(axis=1) / len(features_facilities)


In [10]:
train_X.shape,  train_y.shape, type(train_y), test.shape, type(test)

((411699, 20),
 (411699,),
 pandas.core.series.Series,
 (240224, 20),
 pandas.core.frame.DataFrame)

In [11]:
#Initial casting of features to reduce mem_size
int_features = [ col for col in train_X.columns  if train_X[col].nunique() <50]

In [12]:
#Cast the fearures to the best data type
for df in dataset:
    for col in int_features: 
        df[col] = df[col].astype('int16')

In [13]:
#Drop columns
#drop_features = []
drop_features = ['low_fat', 'gross_weight', 'recyclable_package', 'units_per_case', 'unit_sales(in millions)', "id"]

In [14]:
#List of features

#original_features
#int_features 
#target
all_features = list(train_X.columns)
drop_features = ['low_fat', 'gross_weight', 'recyclable_package', 'units_per_case', 'unit_sales(in millions)', "id"]
features_used = [col for col in all_features  if  col not in drop_features]


In [15]:
result_list = []
#ids = {}
#Cross-validation function for regression and a metric of RMSE_log
def score_model(model, features_used, label=None):
    """Cross-validate a model with feature selection"""
    start_time = datetime.datetime.now()
    score_list = []
    
    global oof
    oof = np.zeros_like(train_y, dtype=float)
    kf = KFold(shuffle=True, random_state=42)
    
    ids = {}
    for fold, (idx_tr, idx_va) in enumerate(kf.split(train_X)):
        X_tr = train_X.iloc[idx_tr][features_used]
        X_va = train_X.iloc[idx_va][features_used]
        y_tr = train_y.iloc[idx_tr]
        y_va = train_y.iloc[idx_va]
        
        #fit the model
        model.fit(X_tr, y_tr)
            
        y_val_pred = model.predict(X_va)
                                            
        valid_score =  mean_squared_log_error(y_val_pred,y_va,squared=False )
        
        score_list.append(valid_score)
        ids[fold] = idx_tr
        execution_time = datetime.datetime.now() - start_time                                    
        print(f"Fold {fold}: RMSE_log = {valid_score:.4f}")                                   
                
    RMSE_mean = np.mean(score_list)                                        
    
    print(f"Meam RMSE_log = {RMSE_mean:.4f}")
    
    if label is not None:
        global result_list
        result_list.append((label, model, RMSE_mean, oof, execution_time, ids))
    

In [16]:
#From public notebooks
xgb_params = {
            'n_estimators': 100,
            'learning_rate': 0.05,
            'max_depth': 8,
            'subsample': 1.0,
            'colsample_bytree': 1.0,
            'n_jobs': -1,
            'objective': 'reg:squarederror',
            'verbosity': 0,
            'eval_metric': 'rmse',
            'random_state': 42}#,
            #"tree_method2" : 'gpu_hist'}

lgb_params = {
            'n_estimators': 100,
            'max_depth': 8,
            'learning_rate': 0.05293702575527996,
            'subsample': 0.20851841295589477,
            'colsample_bytree': 0.5784778854092203,
            'reg_alpha': 0.2622912287429849,
            'reg_lambda': 2.8702494234117617e-08,
            'objective': 'regression',
            'metric': 'rmse',
            'boosting_type': 'gbdt',
            'device': "CPU",
            'random_state': 42
        }

cb_params = {
            'iterations': 100,
            'depth': 7,
            'learning_rate': 0.12947105266151432,
            'l2_leaf_reg': 0.6169164517797081,
            'random_strength': 0.21235850198764036,
            'max_bin': 212,
            'od_wait': 67,
            'one_hot_max_size': 73,
            'grow_policy': 'Depthwise',
            'bootstrap_type': 'Bayesian',
            'od_type': 'Iter',
            'loss_function': 'RMSE',       
            'task_type': "CPU",
            'random_state': 42
        }

In [17]:
lin_reg = LinearRegression()

lgbm = LGBMRegressor(**lgb_params)

xgb = XGBRegressor(**xgb_params)

In [18]:
score_model(lin_reg, features_used, label="Lin_reg")

Fold 0: RMSE_log = 0.3178
Fold 1: RMSE_log = 0.3169
Fold 2: RMSE_log = 0.3174
Fold 3: RMSE_log = 0.3164
Fold 4: RMSE_log = 0.3175
Meam RMSE_log = 0.3172


In [ ]:
score_model(lgbm, features_used, label="lgbm")

In [ ]:
score_model(xgb, features_used, label="xgb")

In [ ]:
result_list = [(a, b, c, d, e, f) for (a, b, c, d, e, f) in result_list if 'Mean' not in a and 'ensemble' not in a]

In [ ]:
result_list

In [ ]:
result_df = pd.DataFrame(result_list, columns=['label', 'model', 'RMSE_log', 'oof', 'execution_time', "ids"])

In [ ]:
result_df

In [ ]:
len(result_df.ids)

In [ ]:
result_df.drop_duplicates(subset='label', keep='last', inplace=True)
result_df = result_df[~result_df.label.str.contains('Mean')]
result_df.sort_values('RMSE_log', inplace=True)
result_df.reset_index(drop=True, inplace=True)

In [ ]:
result_df = result_df.set_index('label')

In [ ]:
result_df

In [ ]:
#This is a filtering step tp remove models that should not be used in the ensemble
labels = ['R GAM', 'Poly3 LogisticRegression', 'xgb', 'HGB',
          'GB', 'LGBM', 'Dart', 'Poly XGB', 'CB']
labels = [l for l in labels if l in r.index]

In [ ]:
print('Ensemble members:', labels)

In [ ]:
oof = np.column_stack(list(r.oof.iloc[:2]))
oof = oof.mean(axis=1)

In [ ]:
for i in range(2, len(r)+1):
    oof = np.column_stack(list(r.oof.iloc[:i]))
    oof = oof.mean(axis=1)
    RMSE_log = mean_squared_log_error(train[target], oof)
    result_list.append((f"Mean of {i}", None, RMSE_log , oof, datetime.timedelta(), 'mean'))

In [ ]:
oof.shape

In [ ]:
ids = {}
kf = KFold(shuffle=True, random_state=42)    
for fold, (idx_tr, idx_va) in enumerate(kf.split(train_X)):
    print(fold)
    ids.update({fold: idx_tr})

In [ ]:
ids

In [ ]:
ads = {}
kf = KFold(shuffle=True, random_state=42)    
for fold, (idx_tr, idx_va) in enumerate(kf.split(train_X)):
    print(fold)
    ads[fold] = idx_tr

In [ ]:
ads